### Step1: Create Python notebook and install libraries

In [1]:
USER_FLAG = "--user"

In [2]:
!pip3 install {USER_FLAG} google-cloud-aiplatform==1.0.0 --upgrade
!pip3 install {USER_FLAG} kfp google-cloud-pipeline-components==0.1.1 --upgrade

     |████████████████████████████████| 1.8 MB 7.8 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
     |████████████████████████████████| 243 kB 9.8 MB/s eta 0:00:01
     |████████████████████████████████| 127 kB 70.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 50.0 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 36 kB/s s eta 0:00:01
     |████████████████████████████████| 54 kB 3.7 MB/s  eta 0:00:01
     |████████████████████████████████| 52 kB 1.9 MB/s  eta 0:00:01
     |████████████████████████████████| 82 kB 1.4 MB/s  eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 87 kB 6.5 MB/s  eta 0:00:01
  Created wheel for kfp: filename=kfp-1.8.0-py3-none-any.whl size=337231 sha256=6ed5d7da7702d52c47c68fd3a2ea0225adda0a23d4305b9d8a7a2d950162bafb

In [3]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [1]:
!python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
!python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

KFP SDK version: 1.8.0
google_cloud_pipeline_components version: 0.1.1


### Step 2: Set the project ID and bucket

In [2]:
import os
PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  project-daisuke-318402


In [3]:
BUCKET_NAME="gs://" + PROJECT_ID + "-bucket"

### Step3: Import libraries

In [4]:
from typing import NamedTuple

import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, ClassificationMetrics, Metrics, component)
from kfp.v2.google.client import AIPlatformClient

from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip

### Step 4: Define Constants

In [5]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin
REGION="us-central1"

PIPELINE_ROOT = f"{BUCKET_NAME}/pipeline_root/"
PIPELINE_ROOT

env: PATH=/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


'gs://project-daisuke-318402-bucket/pipeline_root/'

### Step 5: Creating the first pipeline

- The `@component` decorator compiles this function to a component when the pipeline is run. You'll use this anytime you write a custom component.
- The `base_image` parameter specifies the container image this component will use.
- The `output_component_file` parameter is optional, and specifies the yaml file to write the compiled component to. After running the cell you should see that file written to your notebook instance. If you wanted to share this component with someone, you could send them the generated yaml file and have them load it with the following:

In [6]:
@component(base_image="python:3.9", output_component_file="first-component.yaml")
def product_name(text: str) -> str:
    return text

In [7]:
product_name_component = kfp.components.load_component_from_file('./first-component.yaml')

- The `packages_to_install` paramater tells the component any external library dependencies for this container. In this case, we're using a library called emoji.
- This component returns a `NamedTuple` called `Outputs`. Notice that each of the strings in this tuple have keys: `emoji_text` and `emoji`. We'll use these in our next component to access the output.

In [10]:
@component(packages_to_install=["emoji"])
def emoji(text: str,) -> NamedTuple("Outputs", [("emoji_text", str), ("emoji", str),]):
    import emoji
    emoji_text = text
    emoji_str = emoji.emojize(':' + emoji_text + ':', use_aliases=True)
    print("output one: {}; output_two: {}".format(emoji_text, emoji_str))
    return (emoji_text, emoji_str)

In [16]:
@component
def build_sentence(product: str, emoji: str, emojitext: str) -> str:
    print("We completed the pipeline, hooray!")
    end_str=product+ " is "
    if len(emoji)>0:
        end_str+=emoji
    else:
        end_str+=emojitext
    return(end_str)


The component definitions we defined above created factory functions that can be used in a pipeline definition to create steps. To set up a pipeline, use the `@dsl.pipeline` decorator, give the pipeline a name and description, and provide the root path where your pipeline's artifacts should be written. By artifacts, we mean any output files generated by your pipeline. This intro pipeline doesn't generate any, but our next pipeline will.

In the next block of code we define an `intro_pipeline` function. This is where we specify the inputs to our initial pipeline steps, and how steps connect to each other:

- `product_task` takes a product name as input. Here we're passing "Vertex Pipelines" but you can change this to whatever you'd like.
- `emoji_task` takes the text code for an emoji as input. You can also change this to whatever you'd like. For example, "party_face" refers to the 🥳 emoji. Note that since both this and the product_task component don't have any steps that feed input into them, we manually specify the input for these when we define our pipeline.
- The last step in our pipeline - consumer_task has three input parameters:
  - The output of `product_task`. Since this step only produces one output, we can reference it    via `product_task.output`.
  - The `emoji` output of our emoji_task step. See the emoji component defined above where we named the output parameters.
  - Similarly, the emoji_text named output from the emoji component. In case our pipeline is passed text that doesn't correspond with an emoji, it'll use this text to construct a sentence.

In [17]:
@dsl.pipeline(name="hello-world", description="An intro pipeline", pipeline_root=PIPELINE_ROOT,)

# Change 'Text' and 'emoji_str' parameters here to update the pipeline output
def intro_pipeline(text: str="Vertex Pipelines", emoji_str: str="sparkles"):
    product_task=product_name(text)
    emoji_task=emoji(emoji_str)
    consumer_task=build_sentence(
                                product_task.output,
                                emoji_task.outputs["emoji"],
                                emoji_task.outputs["emoji_text"])

In [18]:
compiler.Compiler().compile(
    pipeline_func=intro_pipeline,
    package_path="intro_pipeline_job.json")

In [20]:
api_client=AIPlatformClient(project_id=PROJECT_ID, region=REGION)

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/google/client/client.py:173: FutureWarning: AIPlatformClient will be deprecated in v1.9. Please use PipelineJob https://googleapis.dev/python/aiplatform/latest/_modules/google/cloud/aiplatform/pipeline_jobs.html in Vertex SDK. Install the SDK using "pip install google-cloud-aiplatform"
  category=FutureWarning,


In [22]:
response=api_client.create_run_from_job_spec(
    job_spec_path="intro_pipeline_job.json",
    # pipeline_root=PIPELINE_ROOT  # this argument is necessary if you did not specify PIPELINE_ROOT as part of the pipeline definition.
)